# Amazon 2014 reveiw dataset from Julian McAuckley team

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from tqdm.auto import tqdm

For this project, I am using the [2014 Amazon book review (5-core) dataset](http://jmcauley.ucsd.edu/data/amazon/) and the book metadata dataset provided by Jim McAuley and his team.  I decided the chose this version given the newer version are larger datasets and given the time frame and capabilites of my laptop, I wanted to use a smaller dataset. 

The reviews and metadata are both provided in zipped JSON files which I will be cleaning, condensing and storing in CSV file in this notebook. Each row in the datasets is a JSON file which makes it ideal to be read in chunkwise given the size of the files. 

I had intially planned to upload to a Postgres database hosted on GCloud, however after condensing the files, it made more sense to leave them as CSVs locally. However if you are working with the larger dataset and would like instructions on how to transfer the files over to GCloud and establishing a connection, click here. 

## Cleaning and condensing the book review dataset

Each row had the following attributes:

- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer
- helpful - helpfulness rating of the review, e.g. \[2,3\] implies the review recieved 2 helpful votes out 3 total votes
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time)
- reviewTime - time of the review (raw)

I decided to drop reviewTime since unixReviewTime captures this information and is more memory efficient as well as the reviewerName as this column was quite messy and wasn't necessary for this project. I converted the helpful column into two columns by sp, helpful_votes and total_votes, as this would be more efficient for EDA and memory. I decided to split the original file into 3 parts as I wanted to be able to do work with the dataframes efficiently when they have been read into memory at the EDA stage and I was working with an 8GM RAM. I ended up only using a portion of the data to build the user-item matrix. However, if you have more capabilites, feel free to use the entire dataset. 

Note: Try to split up the files once the CSV or zipped CSV exceeds 4.3GB. This is because the default MacOS file system will throw an OSError when trying to appending to a file exceeding 4.3GB. 

### Transfer the data from the JSON file to CSV

In [62]:
reader = pd.read_json('/../../../reviews_Books_5.json.gz',
                      compression = 'gzip', lines=True, chunksize=10000)

counter = 0
for df in tqdm(reader, total = 9000000/10000):
    
    df['helpful_votes'] = [x for [x, y] in df.helpful]
    df['total_votes'] = [y for [x, y] in df.helpful]
    
    df.drop(columns=['reviewTime', 'helpful', 'reviewerName'], inplace=True)
    
    if counter <= 3000000:
        df.to_csv('/../../../reviews1.csv.gz',
                          header=False, index=False, mode='a', compression = 'gzip')
    elif counter <= 6000000:
        df.to_csv('../../../reviews2.csv.gz',
                          header=False, index=False, mode='a', compression = 'gzip')
    else:
        df.to_csv('../../../reviews3.csv.gz',
                          header=False, index=False, mode='a', compression = 'gzip')        
    
    counter+=10000
    print(counter)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

### Quick overview of the three files

As mentioned before, I ended up only using the subset of the data. I did this by first taking an overview of the 3 csv files I had created and check their "density", essentially the mean number of reviews per reviewer and book. I picked the most dense file and condensed down further filtering first for all the books that had alteast 50 reviews and then all reviewers who had written 50 reviews. 

#### Part 1

In [3]:
#Dropping all of the columns that are unnecessary for this stage
#Downcasting the rating column for memory efficiency

df_1 = pd.read_csv('/../../../../reviews1.csv.zip', compression = 'zip',
                 names=['reviewerId', 'asin', 'review', 'rating',
                        'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .drop(columns=['review', 'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .astype({'rating': 'int8'})

(Delayed('int-07f88f49-12f6-4e2f-8a7c-158e032be1d4'), 6)

In [3]:
df_1.head()

,reviwerId,asin,rating
0,A10000012B7CGYKOMPQ4L,000100039X,5
1,A2S166WSCFIFP5,000100039X,5
2,A1BM81XB4QHOA3,000100039X,5
3,A1MOSTXNIO5MPJ,000100039X,5
4,A2XQ5LZHTD4AFT,000100039X,5


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010000 entries, 0 to 3009999
Data columns (total 3 columns):
reviwerId    object
asin         object
rating       int8
dtypes: int8(1), object(2)
memory usage: 48.8+ MB


In [5]:
print('Reviewers under 5: ', (df_1.reviwerId.value_counts() < 5).mean())
print('Mean reviewers: ', df_1.reviwerId.value_counts().mean())
print('Median reviewers: ', df_1.reviwerId.value_counts().median())
print('-----------------------------------------------')
print('Books under 5: ', (df_1.asin.value_counts() < 5).mean())
print('Mean books: ', df_1.asin.value_counts().mean())
print('Median books: ', df_1.asin.value_counts().median())

Reviewers under 5:  0.6687836133393875
Mean reviewers:  6.074365571868221
Median reviewers:  3.0
-----------------------------------------------
Books under 5:  0.0
Mean books:  32.62978741856104
Median books:  12.0


#### Part 2

In [3]:
#Dropping all of the columns that are unnecessary for this stage
#Downcasting the rating column for memory efficiency

df_2 = pd.read_csv('/../../../../reviews2.csv.zip', compression = 'zip',
                 names=['reviewerId', 'asin', 'review', 'rating',
                        'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .drop(columns=['review', 'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .astype({'rating': 'int8'})

(Delayed('int-07f88f49-12f6-4e2f-8a7c-158e032be1d4'), 6)

In [4]:
df_2.head()

,reviewerID,asin,rating,unixTime,helpful_votes,overall_votes
0,A10000012B7CGYKOMPQ4L,000100039X,5,1355616000,0,0
1,A2S166WSCFIFP5,000100039X,5,1071100800,0,2
2,A1BM81XB4QHOA3,000100039X,5,1390003200,0,0
3,A1MOSTXNIO5MPJ,000100039X,5,1317081600,0,0
4,A2XQ5LZHTD4AFT,000100039X,5,1033948800,7,9


In [8]:
df_2.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, reviewerID to overall_votes
dtypes: object(2), int32(1), int8(3)

#### Part 3

In [3]:
#Dropping all of the columns that are unnecessary for this stage
#Downcasting the rating column for memory efficiency

df_3 = pd.read_csv('/../../../../reviews3.csv.zip', compression = 'zip',
                 names=['reviewerId', 'asin', 'review', 'rating',
                        'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .drop(columns=['review', 'summary', 'unixtime', 'pos_votes', 'total_votes']) \
    .astype({'rating': 'int8'})

(Delayed('int-07f88f49-12f6-4e2f-8a7c-158e032be1d4'), 6)

In [4]:
df_3.head()

,reviewerID,asin,rating,unixTime,helpful_votes,overall_votes
0,A10000012B7CGYKOMPQ4L,000100039X,5,1355616000,0,0
1,A2S166WSCFIFP5,000100039X,5,1071100800,0,2
2,A1BM81XB4QHOA3,000100039X,5,1390003200,0,0
3,A1MOSTXNIO5MPJ,000100039X,5,1317081600,0,0
4,A2XQ5LZHTD4AFT,000100039X,5,1033948800,7,9


In [8]:
df_3.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, reviewerID to overall_votes
dtypes: object(2), int32(1), int8(3)

In [ ]:
print('Density stats for part 3 before condensing:')
print('Reviewers under 5: ', (df_3.reviewerId.value_counts() < 5).mean())
print('Mean reviewers: ', df_3.reviewerId.value_counts().mean())
print('Median reviewers: ', df_3.reviewerId.value_counts().median())
print('-----------------------------------------------')
print('Books under 5: ', (df_3.asin.value_counts() < 5).mean())
print('Mean books: ', df_3.asin.value_counts().mean())
print('Median books: ', df_3.asin.value_counts().median())

### Condensing the file

I decided to go with part 1 file as it's density stats are marginally better. I still condense the dataset down even further by filtering first for all the books that had alteast 50 reviews and then all reviewers who had written 50 reviews. 

In [10]:
#Filter the number of reviews for each book first
asin_counts = df_1.asin.value_counts()
df_sub = df_1[df_1.asin.isin(asin_counts[asin_counts>50].index)]

#Then filter the number of reviews for each reviewer
review_counts = df_sub.reviewerId.value_counts()
df_sub = df_sub[df_sub.reviewerId.isin(review_counts[review_counts>50].index)]

In [13]:
df_sub = df_sub[df_sub.reviewerId.isin(review_counts[review_counts>50].index)]

In [14]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273661 entries, 1 to 2728290
Data columns (total 5 columns):
reviewerId     273661 non-null object
asin           273661 non-null object
rating         273661 non-null int8
summary        273661 non-null object
total_votes    273661 non-null int32
dtypes: int32(1), int8(1), object(3)
memory usage: 9.7+ MB


Write the resulting dataframe into a csv file to use later on. 

In [15]:
df_sub.to_csv('/../../../df_sub.csv.gz', header=True, index =False, compression='gzip')

## Cleaning and condensing the metadata

Each row for the metadata had the following attributes:

- asin - ID of the product, e.g. 0000031852
- title - name of the product
- price - price in US dollars (at time of crawl)
- imUrl - url of the product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- salesRank - sales rank information
- brand - brand name
- categories - list of categories the product belongs to

In [59]:
reader = pd.read_json('/../../../meta_Books.json.gz', lines=True, chunksize=1000)

counter = 0
for df in tqdm(reader, total=3000000/1000):
    df.price = df.price.str.replace("$", "").str.replace(",", "")
    df.price = df.price.astype('float16')
    df = df[df.title.str.len()< 1000]
    df.to_csv('/Volumes/external/Sangeetha-Project/meta_book.csv', 
              columns = ['asin', 'title', 'description', 'price', 'category'],
                          header=False, mode = 'a', index=False)
    counter+=1000
    print(counter)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

1164000
1165000
1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000


2189000
2190000
2191000
2192000
2193000
2194000
2195000
2196000
2197000
2198000
2199000
2200000
2201000
2202000
2203000
2204000
2205000
2206000
2207000
2208000
2209000
2210000
2211000
2212000
2213000
2214000
2215000
2216000
2217000
2218000
2219000
2220000
2221000
2222000
2223000
2224000
2225000
2226000
2227000
2228000
2229000
2230000
2231000
2232000
2233000
2234000
2235000
2236000
2237000
2238000
2239000
2240000
2241000
2242000
2243000
2244000
2245000
2246000
2247000
2248000
2249000
2250000
2251000
2252000
2253000
2254000
2255000
2256000
2257000
2258000
2259000
2260000
2261000
2262000
2263000
2264000
2265000
2266000
2267000
2268000
2269000
2270000
2271000
2272000
2273000
2274000
2275000
2276000
2277000
2278000
2279000
2280000
2281000
2282000
2283000
2284000
2285000
2286000
2287000
2288000
2289000
2290000
2291000
2292000
2293000
2294000
2295000
2296000
2297000
2298000
2299000
2300000
2301000
2302000
2303000
2304000
2305000
2306000
2307000
2308000
2309000
2310000
2311000
2312000
2313000


In [57]:
df[df.title.str.len()< 1000]

,title,rank,details,main_cat,asin,category,also_buy,brand,also_view,description,price,feature,image
2935000,Vialumi Women's Junior Plus Solid Hooded Kanga...,"4,968,185inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HC3E7ZQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935001,Kikkoman Naturally Brewed Less Sodium Soy Sauc...,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HC3FMKK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935003,Tiger: A Modern Study of Fgst. NR. 250031,"3,119,474inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HC448Z4,NaN,"[099356464X, 6155583137, 1906537313, 398169081...",Lee Lloyd,"[099356464X, 0955642205, 0764310380, 6155583137]","[Lloyd, Balkwill & Johnson; a complete photo s...",NaN,NaN,NaN
2935004,65 year old Gifts 65th Birthday Gifts for All ...,"12,165,524inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HC4I102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935006,Time Commemorative Edition - Prince an Artist ...,"1,501,075inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HC4ZXD0,NaN,NaN,Various,[1683304241],"[Exclusive tributes by Sheila E.,Lenny Kravitz...",14.25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935517,"Galaxy Note 5 Case,Luxury Stylish Design Elect...","6,041,710inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HJA2TAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935518,Colorful Black And Gold Throw 18*18 pillow Case,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HJA02T2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935519,Zhanzy Australia Flag Map Men's Women's Unisex...,"12,981,948inBooks(","\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HJBZV74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935522,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Cas...",NaN,"\n <div class=""content"">\n\n\n\n\n\n\n\n<ul>\...",Books,B01HJEB422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
data = []
counter = 0
row_count = 0

with gzip.open('/Users/Sangeetha/GA/meta_Books.json.gz') as f:
    for l in f:
        #To limit the number of rows
        if counter < 2000:
            data.append(json.loads(l.strip()))
            counter+=1
            row_count+=1
        else: 
            break

    df = pd.DataFrame.from_dict(data)
    df.price = df.price.str.replace("$", "").str.replace(",", "")
    df.price = df.price.astype('float16')
    df.drop(columns=['also_buy', 'brand', 'rank', 'also_view', 'main_cat','image'], inplace=True)